# Author Name Recon Part 2

Following part 2, there are three main cases of ambiguity that can be resolved by affiliations and other methods, we do so here using a number of strategies. Given the disjointed nature of the affiliations data, I will also use this process to iteratively consolidate the two sets of affiliations for common 

This part combines

## Three cases of ambiguity:
1. same name, different person
   - is in same time period -> require affiliations
   - not in same time period -> resolve with significant significant publishing year barrier
2. similar name or alias, same person
   - has multiple resolution from no middle name to has middle name: require affiliations
   - spelling error possibilities. Direct correction in data - require research on individual articles for resolution, restricted to this data set only
3. has multiple resolutions from contraction: require affiliations
   - generate list of contractions and associated possibilities if full name exists


Notes
We only care about authors of articles (any not misc or reviews) between 1940 and 2020
We can only process those articles with affiliations recorded or explicitly set to none

In [295]:
import pandas as pd
# from unidecode import unidecode
import re
from datetime import date
import json
import numpy as np
import string
import time
# from fuzzywuzzy import fuzz 
# from fuzzywuzzy import process 
# set column options
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [296]:
base_path="/Users/sijiawu/Work/Thesis/Data/Affiliations/"

In [297]:
with open(base_path+"author_proc_w_aff.json") as f: 
    data = f.read() 

proc_auths_all = json.loads(data) 

aff_sub=pd.read_pickle(base_path+"affiliations_combined_sub.pkl")

In [298]:
content_ex=['MISC',  'Discussion', 'Review', 'Review2',"Errata"]
content=['Article', 'Comment', 'Reply', 'Rejoinder']
jid=["aer","ecta","jpe","qje","res"]

In [299]:
aff_sub.head()

,id,year,author,auth_num,aff_num,aff_cleaner,aff_cleaner_final,aff_main_final,aff_subunit_final,aff_department_final,alt_final,manual,screened,country
0,1825911,1940,Otto Weinberger,0,1,NaN,NaN,vienna austria (city),NaN,NaN,NaN,NA,Checked,Austria
1,1825513,1940,B. Chait,0,1,NaN,NaN,antwerp (city),NaN,NaN,NaN,MTURK,y,Belgium
2,1883329,1940,Karl H. Niebyl,0,1,NaN,NaN,carleton university,NaN,NaN,NaN,Scopus,NA,Canada
3,1909098,1940,René Roy,0,1,NaN,NaN,universite paris 1 pantheon-sorbonne,NaN,NaN,NaN,MTURK,NA,France
4,1814510,1940,Frederick Pollock,0,1,NaN,NaN,international institute of social research,NaN,NaN,NaN,Manual from PDF,y,International


In [300]:
a1=[]
a2=[]
a3=[]
reduced={}
condense={}
f=[]
d=[]
o=[]
e=[]
k=0
for i in proc_auths_all.keys():
    if (proc_auths_all[i]["content_type"] in content)&(proc_auths_all[i]["year"]>=1940):
            reduced[i]=proc_auths_all[i]
            for j in proc_auths_all[i]['authors'].keys():
                temp=proc_auths_all[i]['authors'][j]
                a1.append(temp['a1'])
                a2.append(temp['a2'])
                a3.append(temp['a3'])
                mains=[]
                if len(temp['affs'])>0:
                    for l in temp["affs"].keys():
                        mains.append(temp["affs"][l]["aff_main_final"])
                reduced[i]['authors'][j]["affs"]=set(mains)    
                condense[k]={"auth_ord":j, "a1":temp['a1'], "a2": temp["a2"], "a3": temp["a3"], "last": temp['a1'].split(" ")[-1],"affs": set(mains), "year":proc_auths_all[i]["year"],"content_type":proc_auths_all[i]["content_type"],"jid":proc_auths_all[i]["jid"], "url": i}
                k=k+1

In [301]:
list(proc_auths_all.keys())[0]


'https://doi.org/10.1093/qje/qjaa012'

In [302]:
proc_auths_all['https://doi.org/10.1093/qje/qjaa012']

{'authors': {'0': {'raw': 'Enke, Benjamin',
   'init': 'benjamin enke',
   'auth_suffix': [],
   'a1': 'benjamin enke',
   'a2': 'b. enke',
   'a3': 'b. enke',
   'affs': {'harvard university',
    'national bureau of economic research - nber'}}},
 'year': 2020,
 'content_type': 'Article',
 'jid': 'qje'}

# Approach
1. At a1, problem of are these two authors the same person despite having the same spelling?
2. At a2, are these two the same person despite having different spelling originally?
3. At a3, are these two the same person despite having different spelling originally?
4. At a3, if author names only result in a unique author




In [303]:
proc_auth_flat=pd.DataFrame(condense).transpose()


In [304]:
uni=list(set(a1))
uni2=list(set(a2))
uni3=list(set(a3))
proc_auth_flat['a1_order']=None
proc_auth_flat['a2_order']=None
proc_auth_flat['a3_order']=None
for i in proc_auth_flat.index:
    index = uni.index(proc_auth_flat.loc[i, "a1"])
    index2 = uni2.index(proc_auth_flat.loc[i, "a2"])
    index3 = uni3.index(proc_auth_flat.loc[i, "a3"])

    proc_auth_flat.loc[i, 'a1_order']=index
    proc_auth_flat.loc[i, 'a2_order']=index2
    proc_auth_flat.loc[i, 'a3_order']=index3

proc_auth_flat['fl']=proc_auth_flat['a1'].str.split(' ').str[0].str.strip() +" " +proc_auth_flat['a1'].str.split(" ").str[-1].str.strip()


In [305]:
proc_auth_flat['a1_tk_count']=proc_auth_flat['a1'].str.split(' ').apply(len)

In [306]:
proc_auth_flat['a1_tk_count'].value_counts()

a1_tk_count
2     24350
3     21357
4       679
5        45
1         2
6         2
10        1
Name: count, dtype: int64

In [307]:
proc_auth_flat.to_pickle(base_path+"proc_auth_aff_flat.pkl")

In [308]:
proc_auth_flat.dtypes

auth_ord        object
a1              object
a2              object
a3              object
last            object
affs            object
year            object
content_type    object
jid             object
url             object
a1_order        object
a2_order        object
a3_order        object
fl              object
a1_tk_count      int64
dtype: object

In [309]:

# a1_r the unique cases all the way from a1 to a3
# a1_s unique cases within a1 and a2 where first names distinguish them out
# a1_t some ambiguity with first name and last name the same but either different or missing middle names
# a1_u if all same token length, we can just check the middle names against each other
# a1_v 

In [310]:
# len(proc_auth_flat[proc_auth_flat['r_a1']==-1]['a1'].unique())


In [311]:
def get_tk_dict(num):
    dict={}
    for i in range(2,num+1,1):
        dict[i]={}
    return dict

def get_a2(name):
    temp=name.split(' ')
    out_name=""
    for i in range(len(temp)-1):
        out_name+=temp[i][0]+ ". "
    return out_name+temp[-1]



In [312]:
# proc_auth_flat[proc_auth_flat['p_a1'].isna()==False].head(5)

In [313]:
# let's for now ignore the fact two entries of anderson a. cooper at the a1 stage may be different people
# The first goal is to get this to a point that we can say a2 and a3 name variations no longer exist in a1
# a2 and a3 variations do not inform on a1 variations
# The trick is to start at the point that the names themselves no longer offer any information as to whether two entries are the same author

# Step 1: separate out all unique a1 names and find the a1 names that need to be matched against that unique a1 set
# use only inherent name properties, no affiliations yet


# strat: assuming if you were to resolve a2 and a3 to a1 format names
# best case?:
# jaegen aasness has a1 of jaegen aasness and a2 of j. aasness and a3 of j. aasness and importantly there are no variations in a1 of jaegen aasness
# if jaegen aasness is the only j. aasness that means if we were to group all the a3 names that match j. aasness 
# also accounts for the case if j. aasness is already at a1 and no a2 or a3 ambiguity exists.
# they will only have one id of jaegen aasness at a1 with and his affiliations will all intersect. 
# These just have the issue that they may be same name, different people at the a1 stage. so lets record their a1 names and orders

# let's take the better case no a2 and a3 formats in a1 - no one decided to contract their name:
# tore olsen and trond e. olsen at a1 resolve to t. olsen at a2 and t. e. olsen at a2, and t. olsen at a3 respectively.
# this will account for the andrew cain and anders cain case as well
# their first names are sufficiently different to tell them apart that they are different people. but their full first name needs to be present in the a1 format first.
# ie: given the same a3 in a set, if a3 or a2 variations do not occur in a1's of this set and all first names are in full then two differing a1's are distinct.
# now, in this set we will have for example - henry j. aaron, henry aaron and harry aaron and harry k. aaron and harry a. aaron and hamlin aaron:
    # group harry, henry and hamlin separately because they are distinct
    # within each group:
        # hamlin is fine and we assign him off
        # harry's - harry k. aaron and harry a. aaron are distinct and can be assigned if you check the middle name contractions, 
        # however, harry aaron is unassignable so hold to be matched for later - include the a1 potential match ids
        # similarly, henry aaron needs to be distinguished from henry j. aaron so hold to be matched for later with the associated potential match id

# the worst case scenario
# suppose henry j. aaron, h. j. aaron, h. aaron and henry k. aaron and harry aaron all exist and let's add henry j. k. aaron
# the only thing we know for certain is that henry j. k. aaron is unique at a1
# henry j. aaron and henry k. aaron could be a variation of it or a different person
# harry aaron could be h. j. aaron
# h. j. aaron is likely a variation of henry j. aaron or henry j. aaron
# h. aaron is likely a variation of henry j. aaron h. j. aaron henry k. aaron harry aaron

proc_auth_flat["r_a1"]=-1
proc_auth_flat["p_a1"]=pd.Series(dtype='object')
c={
    'a1_r':0,
    'a1_s':0,
    'a1_t':0,
    'a1_u':0,
    'a1_v':0,
    'a1_x':0
}
for i in range(len(uni3)):
    # print(uni3[i])
    a3_set=proc_auth_flat[proc_auth_flat['a3_order']==i]
    a1_set=list(set(a3_set['a1_order']))
    a1_n_set=set(a3_set['a1'])
    a2_n_set=set(a3_set['a2'])
    a3_n_set=set(a3_set['a3'])
    
    if len(a1_set)==1:
        # print(a1_set[0])
        proc_auth_flat.loc[proc_auth_flat['a3_order']==i, 'r_a1']=a1_set[0]
        c['a1_r']=c['a1_r']+1
    else:
        fl_set=set(a3_set['fl'])
        # there is no first name contrations to match only middle name contractions
        if (len(a1_n_set.intersection(a2_n_set))==0) & (len(a1_n_set.intersection(a3_n_set))==0):
            # print(a1_n_set)
            # print(a1_n_set)
            # print(fl_set)
            for j in fl_set:
                fl_subset=proc_auth_flat[proc_auth_flat['fl']==j]
                fl_set_red=set(fl_subset['a1'])
                if len(fl_set_red)==1:
                    proc_auth_flat.loc[proc_auth_flat['fl']==1, 'r_a1']=list(set(fl_subset['a1_order']))[0]
                    c['a1_s']=c['a1_s']+1                    
                else:
                    # print(j)
                    tk_set=set(fl_subset['a1_tk_count'])
                    if len(tk_set)==1:
                        for k in fl_set_red:
                            # print(k)
                            proc_auth_flat.loc[proc_auth_flat['a1']==k, 'r_a1']=list(set(proc_auth_flat[proc_auth_flat['a1']==k]['a1_order']))[0]
                        c['a1_u']=c['a1_u']+1
                    else:
                        tk_org=get_tk_dict(max(tk_set))
                        tk_all={}
                        tk_set_l=list(tk_set)
                        tk_set_l.sort(reverse=True)
                        max_tk=max(tk_set_l)
                        min_tk=min(tk_set_l)

                        for k in fl_set_red:
                            tk_org[len(k.split(' '))][k[len(k.split(' ')[0]):-len(k.split(' ')[-1])].strip()]=k
                            tk_all[k[len(k.split(' ')[0]):-len(k.split(' ')[-1])].strip()]=k
                        # print(tk_org)
                        
                        mx=[]
                        # if anything upwards has 
                        for k in tk_set_l:
                            # print(k)
                            for l in tk_org[k]:
                                pot=[]
                                for m in mx:
                                    if l in m:
                                        pot.append(m)
                                if len(pot)==0:
                                    mx.append(l)
                                    # print(tk_org[k][l])
                                    proc_auth_flat.loc[proc_auth_flat['a1']==tk_org[k][l], 'r_a1']=list(set(proc_auth_flat[proc_auth_flat['a1']==tk_org[k][l]]['a1_order']))[0]
                                else:
                                    for n in range(len(pot)):
                                        pot[n]=tk_all[pot[n]]
                                    # print(tk_org[k][l])
                                    # print(pot)
                                    mask=proc_auth_flat['a1']==tk_org[k][l]
                                    # print(np.array(pot))
                                    proc_auth_flat.loc[mask, 'p_a1']=json.dumps(pot)
                        c['a1_v']=c['a1_v']+1
        # groups with first name contracted and first name not contracted        
        else:
            # print(a1_n_set)
            a_a3=a1_n_set.intersection(a3_n_set)
            a_a2=a1_n_set.intersection(a2_n_set)

            m_a1=a1_n_set.intersection(a3_n_set)|a1_n_set.intersection(a2_n_set)
            a_a1=a1_n_set^m_a1

            for j in a_a1:
                print(j)

            for j in m_a1:
                print(j)
            # for j in a1_n_set:
            #     print(j)
            # tk_set=set(a3_set[a3_set["a1"].isin(list(a_a1))]['a1_tk_count'])
            fl_set=set(a3_set['fl'])
            print(fl_set)
            
            print()
    # if i >300:
    #     break



richard manning
r. manning
{'r. manning', 'richard manning'}

andreu mas-colell
a. mas-colell
{'andreu mas-colell', 'a. mas-colell'}

charalambos d. aliprantis
c. d. aliprantis
{'c. aliprantis', 'charalambos aliprantis'}

john m. letiche
j. m. letiche
{'j. letiche', 'john letiche'}

arthur b. treadway
a. b. treadway
{'a. treadway', 'arthur treadway'}

ernest w. adams
e. s. adams
{'e. adams', 'ernest adams'}

steven m. goldman
s. m. goldman
{'s. goldman', 'steven goldman'}

david s. huang
d. s. huang
{'david huang', 'd. huang'}

hal j. singer
h. w. singer
{'hal singer', 'h. singer'}

john wise
j. wise
{'john wise', 'j. wise'}

armen alchian
armen a. alchian
a. a. alchian
{'armen alchian', 'a. alchian'}

anthony f. shorrocks
a. f. shorrocks
{'a. shorrocks', 'anthony shorrocks'}

arja h. turunen-red
a. h. turunen-red
{'a. turunen-red', 'arja turunen-red'}

joshua b. miller
john p. miller
john h. miller
john miller
james c. miller
jon r. miller
jeffrey b. miller
j. p. miller
j. d. miller
{

In [314]:
proc_auth_flat.sort_values(by=["a3","a2","a1","year"])
proc_auth_flat.to_excel(base_path+"auth_proc_w_aff_flat.xlsx", index=False)
proc_auth_flat.to_pickle(base_path+"auth_proc_w_aff_flat.pkl")

with open(base_path+"author_proc_reduced.json", "w") as outfile: 
    json.dump(reduced, outfile, indent=4, default=int)

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'set'